<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/Leer_fastas_Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#download data
!wget https://github.com/adigenova/wengan_demo/raw/master/ecoli/reads/EC.50X.R1.fastq.gz
!wget https://raw.githubusercontent.com/adigenova/wengan_demo/master/ecoli/reference/ecoli-k12-ref.fna

--2023-10-02 02:43:28--  https://github.com/adigenova/wengan_demo/raw/master/ecoli/reads/EC.50X.R1.fastq.gz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/adigenova/wengan_demo/master/ecoli/reads/EC.50X.R1.fastq.gz [following]
--2023-10-02 02:43:28--  https://raw.githubusercontent.com/adigenova/wengan_demo/master/ecoli/reads/EC.50X.R1.fastq.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83923660 (80M) [application/octet-stream]
Saving to: ‘EC.50X.R1.fastq.gz’

EC.50X.R1.fastq.gz  100%[===================>]  80.04M   310MB/s    in 0.3s    

2023-10-02 02:43:28 (310 MB/s) - ‘EC.50X.R1.fastq.g

In [2]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.15_GRCh38/seqs_for_alignment_pipelines.ucsc_ids/GCA_000001405.15_GRCh38_full_analysis_set.fna.gz

--2023-10-02 02:44:18--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.15_GRCh38/seqs_for_alignment_pipelines.ucsc_ids/GCA_000001405.15_GRCh38_full_analysis_set.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 2607:f220:41f:250::228, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 903026760 (861M) [application/x-gzip]
Saving to: ‘GCA_000001405.15_GRCh38_full_analysis_set.fna.gz’

GCA_000001405.15_GR 100%[===================>] 861.19M   152MB/s    in 5.4s    

2023-10-02 02:44:23 (159 MB/s) - ‘GCA_000001405.15_GRCh38_full_analysis_set.fna.gz’ saved [903026760/903026760]



In [45]:
%%writefile kseq.h

#ifndef AC_KSEQ_H
#define AC_KSEQ_H

#include <ctype.h>
#include <string.h>
#include <stdlib.h>

#define KS_SEP_SPACE 0 // isspace(): \t, \n, \v, \f, \r
#define KS_SEP_TAB   1 // isspace() && !' '
#define KS_SEP_LINE  2 // line separator: "\n" (Unix) or "\r\n" (Windows)
#define KS_SEP_MAX   2

#ifndef klib_unused
#if (defined __clang__ && __clang_major__ >= 3) || (defined __GNUC__ && __GNUC__ >= 3)
#define klib_unused __attribute__ ((__unused__))
#else
#define klib_unused
#endif
#endif /* klib_unused */

#define __KS_TYPE(type_t) \
	typedef struct __kstream_t { \
		int begin, end; \
		int is_eof:2, bufsize:30; \
		type_t f; \
		unsigned char *buf; \
	} kstream_t;

#define ks_eof(ks) ((ks)->is_eof && (ks)->begin >= (ks)->end)
#define ks_rewind(ks) ((ks)->is_eof = (ks)->begin = (ks)->end = 0)

#define __KS_BASIC(SCOPE, type_t, __bufsize) \
	SCOPE kstream_t *ks_init(type_t f) \
	{ \
		kstream_t *ks = (kstream_t*)calloc(1, sizeof(kstream_t)); \
		ks->f = f; ks->bufsize = __bufsize; \
		ks->buf = (unsigned char*)malloc(__bufsize); \
		return ks; \
	} \
	SCOPE void ks_destroy(kstream_t *ks) \
	{ \
		if (!ks) return; \
		free(ks->buf); \
		free(ks); \
	}

#define __KS_INLINED(__read) \
	static inline klib_unused int ks_getc(kstream_t *ks) \
	{ \
		if (ks->is_eof && ks->begin >= ks->end) return -1; \
		if (ks->begin >= ks->end) { \
			ks->begin = 0; \
			ks->end = __read(ks->f, ks->buf, ks->bufsize); \
			if (ks->end < ks->bufsize) ks->is_eof = 1; \
			if (ks->end == 0) return -1; \
		} \
		return (int)ks->buf[ks->begin++]; \
	} \
	static inline int ks_getuntil(kstream_t *ks, int delimiter, kstring_t *str, int *dret) \
	{ return ks_getuntil2(ks, delimiter, str, dret, 0); }

#ifndef KSTRING_T
#define KSTRING_T kstring_t
typedef struct __kstring_t {
	unsigned l, m;
	char *s;
} kstring_t;
#endif

#ifndef kroundup32
#define kroundup32(x) (--(x), (x)|=(x)>>1, (x)|=(x)>>2, (x)|=(x)>>4, (x)|=(x)>>8, (x)|=(x)>>16, ++(x))
#endif

#define __KS_GETUNTIL(SCOPE, __read) \
	SCOPE int ks_getuntil2(kstream_t *ks, int delimiter, kstring_t *str, int *dret, int append) \
	{ \
		if (dret) *dret = 0; \
		str->l = append? str->l : 0; \
		if (ks->begin >= ks->end && ks->is_eof) return -1; \
		for (;;) { \
			int i; \
			if (ks->begin >= ks->end) { \
				if (!ks->is_eof) { \
					ks->begin = 0; \
					ks->end = __read(ks->f, ks->buf, ks->bufsize); \
					if (ks->end < ks->bufsize) ks->is_eof = 1; \
					if (ks->end == 0) break; \
				} else break; \
			} \
			if (delimiter == KS_SEP_LINE) { \
				for (i = ks->begin; i < ks->end; ++i) \
					if (ks->buf[i] == '\n') break; \
			} else if (delimiter > KS_SEP_MAX) { \
				for (i = ks->begin; i < ks->end; ++i) \
					if (ks->buf[i] == delimiter) break; \
			} else if (delimiter == KS_SEP_SPACE) { \
				for (i = ks->begin; i < ks->end; ++i) \
					if (isspace(ks->buf[i])) break; \
			} else if (delimiter == KS_SEP_TAB) { \
				for (i = ks->begin; i < ks->end; ++i) \
					if (isspace(ks->buf[i]) && ks->buf[i] != ' ') break; \
			} else i = 0; /* never come to here! */ \
			if (str->m - str->l < (size_t)(i - ks->begin + 1)) { \
				str->m = str->l + (i - ks->begin) + 1; \
				kroundup32(str->m); \
				str->s = (char*)realloc(str->s, str->m); \
			} \
			memcpy(str->s + str->l, ks->buf + ks->begin, i - ks->begin); \
			str->l = str->l + (i - ks->begin); \
			ks->begin = i + 1; \
			if (i < ks->end) { \
				if (dret) *dret = ks->buf[i]; \
				break; \
			} \
		} \
		if (str->s == 0) { \
			str->m = 1; \
			str->s = (char*)calloc(1, 1); \
		} else if (delimiter == KS_SEP_LINE && str->l > 1 && str->s[str->l-1] == '\r') --str->l; \
		str->s[str->l] = '\0'; \
		return str->l; \
	}

#define KSTREAM_INIT2(SCOPE, type_t, __read, __bufsize) \
	__KS_TYPE(type_t) \
	__KS_BASIC(SCOPE, type_t, __bufsize) \
	__KS_GETUNTIL(SCOPE, __read) \
	__KS_INLINED(__read)

#define KSTREAM_INIT(type_t, __read, __bufsize) KSTREAM_INIT2(static, type_t, __read, __bufsize)

#define KSTREAM_DECLARE(type_t, __read) \
	__KS_TYPE(type_t) \
	extern int ks_getuntil2(kstream_t *ks, int delimiter, kstring_t *str, int *dret, int append); \
	extern kstream_t *ks_init(type_t f); \
	extern void ks_destroy(kstream_t *ks); \
	__KS_INLINED(__read)

/******************
 * FASTA/Q parser *
 ******************/

#define kseq_rewind(ks) ((ks)->last_char = (ks)->f->is_eof = (ks)->f->begin = (ks)->f->end = 0)

#define __KSEQ_BASIC(SCOPE, type_t) \
	SCOPE kseq_t *kseq_init(type_t fd) \
	{ \
		kseq_t *s = (kseq_t*)calloc(1, sizeof(kseq_t)); \
		s->f = ks_init(fd); \
		return s; \
	} \
	SCOPE void kseq_destroy(kseq_t *ks) \
	{ \
		if (!ks) return; \
		free(ks->name.s); free(ks->comment.s); free(ks->seq.s);	free(ks->qual.s); \
		ks_destroy(ks->f); \
		free(ks); \
	}

/* Return value:
   >=0  length of the sequence (normal)
   -1   end-of-file
   -2   truncated quality string
 */
#define __KSEQ_READ(SCOPE) \
	SCOPE int kseq_read(kseq_t *seq) \
	{ \
		int c; \
		kstream_t *ks = seq->f; \
		if (seq->last_char == 0) { /* then jump to the next header line */ \
			while ((c = ks_getc(ks)) != -1 && c != '>' && c != '@'); \
			if (c == -1) return -1; /* end of file */ \
			seq->last_char = c; \
		} /* else: the first header char has been read in the previous call */ \
		seq->comment.l = seq->seq.l = seq->qual.l = 0; /* reset all members */ \
		if (ks_getuntil(ks, 0, &seq->name, &c) < 0) return -1; /* normal exit: EOF */ \
		if (c != '\n') ks_getuntil(ks, KS_SEP_LINE, &seq->comment, 0); /* read FASTA/Q comment */ \
		if (seq->seq.s == 0) { /* we can do this in the loop below, but that is slower */ \
			seq->seq.m = 256; \
			seq->seq.s = (char*)malloc(seq->seq.m); \
		} \
		while ((c = ks_getc(ks)) != -1 && c != '>' && c != '+' && c != '@') { \
			if (c == '\n') continue; /* skip empty lines */ \
			seq->seq.s[seq->seq.l++] = c; /* this is safe: we always have enough space for 1 char */ \
			ks_getuntil2(ks, KS_SEP_LINE, &seq->seq, 0, 1); /* read the rest of the line */ \
		} \
		if (c == '>' || c == '@') seq->last_char = c; /* the first header char has been read */ \
		if (seq->seq.l + 1 >= seq->seq.m) { /* seq->seq.s[seq->seq.l] below may be out of boundary */ \
			seq->seq.m = seq->seq.l + 2; \
			kroundup32(seq->seq.m); /* rounded to the next closest 2^k */ \
			seq->seq.s = (char*)realloc(seq->seq.s, seq->seq.m); \
		} \
		seq->seq.s[seq->seq.l] = 0;	/* null terminated string */ \
		if (c != '+') return seq->seq.l; /* FASTA */ \
		if (seq->qual.m < seq->seq.m) {	/* allocate memory for qual in case insufficient */ \
			seq->qual.m = seq->seq.m; \
			seq->qual.s = (char*)realloc(seq->qual.s, seq->qual.m); \
		} \
		while ((c = ks_getc(ks)) != -1 && c != '\n'); /* skip the rest of '+' line */ \
		if (c == -1) return -2; /* error: no quality string */ \
		while (ks_getuntil2(ks, KS_SEP_LINE, &seq->qual, 0, 1) >= 0 && seq->qual.l < seq->seq.l); \
		seq->last_char = 0;	/* we have not come to the next header line */ \
		if (seq->seq.l != seq->qual.l) return -2; /* error: qual string is of a different length */ \
		return seq->seq.l; \
	}

#define __KSEQ_TYPE(type_t) \
	typedef struct { \
		kstring_t name, comment, seq, qual; \
		int last_char; \
		kstream_t *f; \
	} kseq_t;

#define KSEQ_INIT2(SCOPE, type_t, __read) \
	KSTREAM_INIT2(SCOPE, type_t, __read, 16384) \
	__KSEQ_TYPE(type_t) \
	__KSEQ_BASIC(SCOPE, type_t) \
	__KSEQ_READ(SCOPE)

#define KSEQ_INIT(type_t, __read) KSEQ_INIT2(static, type_t, __read)

#define KSEQ_DECLARE(type_t) \
	__KS_TYPE(type_t) \
	__KSEQ_TYPE(type_t) \
	extern kseq_t *kseq_init(type_t fd); \
	void kseq_destroy(kseq_t *ks); \
	int kseq_read(kseq_t *seq);

#endif


Overwriting kseq.h


In [46]:
%%writefile read_fasta.c

#include <stdio.h>
#include <stdint.h>
#include <zlib.h>
//libreria para leer archivos FASTA/FASTQ
#include "kseq.h"

// Iniciamos las funciones de lectura de archivos comprimidos en el caso de ser necesario.
KSEQ_INIT(gzFile, gzread)

int main(int argc, char *argv[])
{
gzFile fp;
kseq_t *ks;
if ((fp = gzopen(argv[1], "r")) == 0) return 0;
ks = kseq_init(fp);
char sseq[12];
printf("%s\n",argv[1]);
int k=10;
int nseq=0;//contador para imprimir 5 secuencias
while (kseq_read(ks) >= 0 && nseq <5){
  strncpy(sseq,ks->seq.s,10);
  //dividimos la secuencia en kmers
  //for(int i=0; i<ks->seq.l-k;i++){
   //imprimimos 20 10-mers
  for(int i=0; i<k*2;i++){
     strncpy(sseq,ks->seq.s+i,10);
    printf("%s %d %s\n",ks->name.s, i, sseq);
  }
	nseq++;
}
kseq_destroy(ks);
gzclose(fp);
return 0;
}


Overwriting read_fasta.c


In [47]:
!gcc read_fasta.c -o read_fasta -g -Wall -O2 -lz

In [48]:
! ./read_fasta /content/ecoli-k12-ref.fna

/content/ecoli-k12-ref.fna
NC_000913.3 0 AGCTTTTCAT
NC_000913.3 1 GCTTTTCATT
NC_000913.3 2 CTTTTCATTC
NC_000913.3 3 TTTTCATTCT
NC_000913.3 4 TTTCATTCTG
NC_000913.3 5 TTCATTCTGA
NC_000913.3 6 TCATTCTGAC
NC_000913.3 7 CATTCTGACT
NC_000913.3 8 ATTCTGACTG
NC_000913.3 9 TTCTGACTGC
NC_000913.3 10 TCTGACTGCA
NC_000913.3 11 CTGACTGCAA
NC_000913.3 12 TGACTGCAAC
NC_000913.3 13 GACTGCAACG
NC_000913.3 14 ACTGCAACGG
NC_000913.3 15 CTGCAACGGG
NC_000913.3 16 TGCAACGGGC
NC_000913.3 17 GCAACGGGCA
NC_000913.3 18 CAACGGGCAA
NC_000913.3 19 AACGGGCAAT


In [49]:
! ./read_fasta /content/EC.50X.R1.fastq.gz

/content/EC.50X.R1.fastq.gz
M00141:217:000000000-A55DC:1:1101:10937:1000 0 NTGAACTCCA
M00141:217:000000000-A55DC:1:1101:10937:1000 1 TGAACTCCAC
M00141:217:000000000-A55DC:1:1101:10937:1000 2 GAACTCCACA
M00141:217:000000000-A55DC:1:1101:10937:1000 3 AACTCCACAT
M00141:217:000000000-A55DC:1:1101:10937:1000 4 ACTCCACATC
M00141:217:000000000-A55DC:1:1101:10937:1000 5 CTCCACATCC
M00141:217:000000000-A55DC:1:1101:10937:1000 6 TCCACATCCT
M00141:217:000000000-A55DC:1:1101:10937:1000 7 CCACATCCTG
M00141:217:000000000-A55DC:1:1101:10937:1000 8 CACATCCTGC
M00141:217:000000000-A55DC:1:1101:10937:1000 9 ACATCCTGCG
M00141:217:000000000-A55DC:1:1101:10937:1000 10 CATCCTGCGG
M00141:217:000000000-A55DC:1:1101:10937:1000 11 ATCCTGCGGA
M00141:217:000000000-A55DC:1:1101:10937:1000 12 TCCTGCGGAT
M00141:217:000000000-A55DC:1:1101:10937:1000 13 CCTGCGGATC
M00141:217:000000000-A55DC:1:1101:10937:1000 14 CTGCGGATCG
M00141:217:000000000-A55DC:1:1101:10937:1000 15 TGCGGATCGT
M00141:217:000000000-A55DC:1:1101:1093

In [50]:
! ./read_fasta /content/GCA_000001405.15_GRCh38_full_analysis_set.fna.gz

/content/GCA_000001405.15_GRCh38_full_analysis_set.fna.gz
chr1 0 NNNNNNNNNN
chr1 1 NNNNNNNNNN
chr1 2 NNNNNNNNNN
chr1 3 NNNNNNNNNN
chr1 4 NNNNNNNNNN
chr1 5 NNNNNNNNNN
chr1 6 NNNNNNNNNN
chr1 7 NNNNNNNNNN
chr1 8 NNNNNNNNNN
chr1 9 NNNNNNNNNN
chr1 10 NNNNNNNNNN
chr1 11 NNNNNNNNNN
chr1 12 NNNNNNNNNN
chr1 13 NNNNNNNNNN
chr1 14 NNNNNNNNNN
chr1 15 NNNNNNNNNN
chr1 16 NNNNNNNNNN
chr1 17 NNNNNNNNNN
chr1 18 NNNNNNNNNN
chr1 19 NNNNNNNNNN
chr2 0 NNNNNNNNNN
chr2 1 NNNNNNNNNN
chr2 2 NNNNNNNNNN
chr2 3 NNNNNNNNNN
chr2 4 NNNNNNNNNN
chr2 5 NNNNNNNNNN
chr2 6 NNNNNNNNNN
chr2 7 NNNNNNNNNN
chr2 8 NNNNNNNNNN
chr2 9 NNNNNNNNNN
chr2 10 NNNNNNNNNN
chr2 11 NNNNNNNNNN
chr2 12 NNNNNNNNNN
chr2 13 NNNNNNNNNN
chr2 14 NNNNNNNNNN
chr2 15 NNNNNNNNNN
chr2 16 NNNNNNNNNN
chr2 17 NNNNNNNNNN
chr2 18 NNNNNNNNNN
chr2 19 NNNNNNNNNN
chr3 0 NNNNNNNNNN
chr3 1 NNNNNNNNNN
chr3 2 NNNNNNNNNN
chr3 3 NNNNNNNNNN
chr3 4 NNNNNNNNNN
chr3 5 NNNNNNNNNN
chr3 6 NNNNNNNNNN
chr3 7 NNNNNNNNNN
chr3 8 NNNNNNNNNN
chr3 9 NNNNNNNNNN
chr3 10 NNNNNNNNNN
chr